# Zero-GRPO Reward Evaluation on Kaggle

This notebook evaluates the multi-view consistency reward function using Ground Truth data rendered from Objaverse models via Blender.

**Pre-requisites**:
- Accelerator: GPU T4 x2 (recommended)
- Internet: On

## 1. Setup & Installation

In [ ]:
# Clone Repository
!git clone https://github.com/kyrozepto/zero-grpo.git
%cd zero-grpo
# !git checkout your-branch-name  # Uncomment if needed

In [ ]:
# Install Dependencies
!pip install -r requirements.txt
!pip install kornia

# Install Blender (Headless) for rendering GT
!apt-get update
!apt-get install blender -y

## 2. Download Sample Data

In [ ]:
import os
os.makedirs("models", exist_ok=True)

# Download sample GLB files (Duck and DamagedHelmet from Khronos)
!wget -O models/example_1.glb https://raw.githubusercontent.com/KhronosGroup/glTF-Sample-Models/master/2.0/Duck/glTF-Binary/Duck.glb
!wget -O models/example_2.glb https://raw.githubusercontent.com/KhronosGroup/glTF-Sample-Models/master/2.0/DamagedHelmet/glTF-Binary/DamagedHelmet.glb

## 3. Render Ground Truth Images

We use the custom Blender script to render 6 views matching Zero123++ v1.2 specifications (FoV 30, Elev 20/-10).

In [ ]:
# Render Object 1
!blender --background --python scripts/render_zero123plus.py -- models/example_1.glb --output_dir data/gt/example_1

# Render Object 2
!blender --background --python scripts/render_zero123plus.py -- models/example_2.glb --output_dir data/gt/example_2

## 4. Evaluate Rewards

Run the `CompositeReward` on the generated images. We expect high scores (> 0.8) since these are perfect 3D consistency renders.

In [ ]:
import torch
import sys
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

# Ensure local modules are importable
sys.path.append('.')
from src.rewards import CompositeReward

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize Reward Function
reward_fn = CompositeReward(device=device)

def load_views(path):
    views = []
    for i in range(6):
        img_path = f"{path}/{i:03d}.png"
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image not found: {img_path}")
        img = Image.open(img_path).convert('RGB')
        views.append(transforms.ToTensor()(img))
    # Stack and add batch dimension -> (1, 6, 3, H, W)
    return torch.stack(views).unsqueeze(0).to(device)

# List of paths to evaluate
sample_paths = ['data/gt/example_1', 'data/gt/example_2']

for path in sample_paths:
    try:
        images = load_views(path)
        
        # Use the first view as the 'conditioning' image
        condition = images[:, 0]
        
        # Compute Rewards
        # Step=10000 ensures maximum geometric weight (curriculum end)
        with torch.no_grad():
            results = reward_fn.forward(images, condition, step=10000)
            
        print(f"\nEvaluating: {path}")
        print(f"Total Reward: {results['total'].item():.4f}")
        print(f"  > Warp Score: {results['warp'].item():.4f}")
        print(f"  > Epi Score:  {results['epipolar'].item():.4f}")
        print(f"  > Sem Score:  {results['semantic'].item():.4f}")
        
        # Visualization
        fig, axes = plt.subplots(1, 6, figsize=(12, 2))
        for i in range(6):
            axes[i].imshow(images[0, i].cpu().permute(1, 2, 0))
            axes[i].axis('off')
        plt.suptitle(f"{path} (R_total: {results['total'].item():.2f})", fontsize=10)
        plt.show()
        
    except Exception as e:
        print(f"Error processing {path}: {e}")